In [3]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm

n = 256
n_train = 10000
n_test = 5000
s_max = 20
step = 0.05

# noise - 0
# ЛЧМ I - 1
# ЛЧМ II - 2

f0_1 = 0.1
f0_2 = 0.1
b_1 = 1
b_2 = 2
s0_1 = 1
s0_2 = 1
T = np.linspace(0, n / 100, n) 
S = np.linspace(0, 20, 401)

train_phi0_0 = norm.rvs(loc=0, scale=1, size=(len(S)*n_train, n)).astype(np.float32)
test_phi0_0 = norm.rvs(loc=0, scale=1, size=(len(S)*n_test, n)).astype(np.float32)
train_phi0_random = norm.rvs(loc=0, scale=1, size=(len(S)*n_train, n)).astype(np.float32)
test_phi0_random = norm.rvs(loc=0, scale=1, size=(len(S)*n_test, n)).astype(np.float32)

dispersia_train = np.zeros(shape=(len(S)*n_train), dtype=np.float32)
dispersia_test = np.zeros(shape=(len(S)*n_test), dtype=np.float32)

signal_type_train = np.zeros(shape=(len(S)*n_train), dtype=np.float32)
signal_type_test = np.zeros(shape=(len(S)*n_test), dtype=np.float32)

for s in S:
    lfm1_train_phi0_0 = np.array([s0_1 * np.cos(2*np.pi*(f0_1 * T + 0.5 * b_1 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_train)]).astype(np.float32)
    lfm2_train_phi0_0 = np.array([s0_2 * np.cos(2*np.pi*(f0_2 * T + 0.5 * b_2 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_train)]).astype(np.float32)
    train_phi0_0 = np.vstack((train_phi0_0, lfm1_train_phi0_0, lfm2_train_phi0_0))
    
    lfm1_test_phi0_0 = np.array([s0_1 * np.cos(2*np.pi*(f0_1 * T + 0.5 * b_1 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_test)]).astype(np.float32)
    lfm2_test_phi0_0 = np.array([s0_2 * np.cos(2*np.pi*(f0_2 * T + 0.5 * b_2 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_test)]).astype(np.float32)
    test_phi0_0 = np.vstack((test_phi0_0, lfm1_test_phi0_0, lfm2_test_phi0_0))

    lfm1_train_phi0_random = np.array([s0_1 * np.cos(np.random.uniform(low=-np.pi, high=np.pi, size=None) + 2*np.pi*(f0_1 * T + 0.5 * b_1 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_train)]).astype(np.float32)
    lfm2_train_phi0_random = np.array([s0_2 * np.cos(np.random.uniform(low=-np.pi, high=np.pi, size=None) + 2*np.pi*(f0_2 * T + 0.5 * b_2 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_train)]).astype(np.float32)
    train_phi0_random = np.vstack((train_phi0_random, lfm1_train_phi0_random, lfm2_train_phi0_random))

    lfm1_test_phi0_random = np.array([s0_1 * np.cos(np.random.uniform(low=-np.pi, high=np.pi, size=None) + 2*np.pi*(f0_1 * T + 0.5 * b_1 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_test)]).astype(np.float32)
    lfm2_test_phi0_random = np.array([s0_2 * np.cos(np.random.uniform(low=-np.pi, high=np.pi, size=None) + 2*np.pi*(f0_2 * T + 0.5 * b_2 * T ** 2)) + norm.rvs(loc=0, scale=s, size=n) for i in range(n_test)]).astype(np.float32)
    test_phi0_random = np.vstack((test_phi0_random, lfm1_test_phi0_random, lfm2_test_phi0_random))

    dispersia_train = np.hstack((dispersia_train, np.array([s for i in range(n_train)]).astype(np.float32), np.array([s for i in range(n_train)]).astype(np.float32)))
    dispersia_test = np.hstack((dispersia_test, np.array([s for i in range(n_test)]).astype(np.float32), np.array([s for i in range(n_test)]).astype(np.float32)))
    
    signal_type_train = np.hstack((signal_type_train, np.ones(shape=(n_train,), dtype=np.uint8), np.ones(shape=(n_train,), dtype=np.uint8)*2))
    signal_type_test = np.hstack((signal_type_test, np.ones(shape=(n_test,), dtype=np.uint8), np.ones(shape=(n_test,), dtype=np.uint8)*2))

with h5py.File('/home/other/signals/data/train_phi0_0.hdf5', 'w') as f:
    f.create_dataset("X", data=train_phi0_0, dtype=np.float32)
    f.create_dataset("dispersia", data=dispersia_train, dtype=np.float32)
    f.create_dataset("Y", data=signal_type_train, dtype=np.uint8)
    
with h5py.File('/home/other/signals/data/test_phi0_0.hdf5', 'w') as f:
    f.create_dataset("X", data=test_phi0_0, dtype=np.float32)
    f.create_dataset("dispersia", data=dispersia_test, dtype=np.float32)
    f.create_dataset("Y", data=signal_type_test, dtype=np.uint8)
    
with h5py.File('/home/other/signals/data/train_phi0_random.hdf5', 'w') as f:
    f.create_dataset("X", data=train_phi0_random, dtype=np.float32)
    f.create_dataset("dispersia", data=dispersia_train, dtype=np.float32)
    f.create_dataset("Y", data=signal_type_train, dtype=np.uint8)

with h5py.File('/home/other/signals/data/test_phi0_random.hdf5', 'w') as f:
    f.create_dataset("X", data=test_phi0_random, dtype=np.float32)
    f.create_dataset("dispersia", data=dispersia_test, dtype=np.float32)
    f.create_dataset("Y", data=signal_type_test, dtype=np.uint8)